# Import Libraries

In [42]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import missingno as msno

from scipy.stats.mstats import winsorize

import re

from bs4 import BeautifulSoup

In [43]:
# import dataset

df = pd.read_csv(r"C:\Users\benke\Downloads\Cleaning Marathon\archive (25)\fake_job_postings.csv")

# Data Investigation

In [44]:
# check first 5 rows

df.head(3)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0


In [45]:
# duplicated

df.duplicated().sum()

0

In [46]:
# info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15184 non-null  object
 8   benefits             10668 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

In [47]:
# missing values

df.isnull().sum()

job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2696
benefits                7212
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [48]:
# satatistical info

df.describe()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent
count,17880.000000,17880.000000,17880.000000,17880.000000,17880.000000
mean,8940.500000,0.042897,0.795302,0.491723,0.048434
std,5161.655742,0.202631,0.403492,0.499945,0.214688
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4470.750000,0.000000,1.000000,0.000000,0.000000
50%,8940.500000,0.000000,1.000000,0.000000,0.000000
75%,13410.250000,0.000000,1.000000,1.000000,0.000000
max,17880.000000,1.000000,1.000000,1.000000,1.000000


# Data Cleaning

In [49]:
# first 5 rows

df.head(20)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
5,6,Accounting Clerk,"US, MD,",NaN,NaN,NaN,Job OverviewApex is an environmental consultin...,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,0
6,7,Head of Content (m/f),"DE, BE, Berlin",ANDROIDPIT,20000-28000,"Founded in 2009, the Fonpit AG rose with its i...",Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Your Benefits: Being part of a fast-growing co...,0,1,1,Full-time,Mid-Senior level,Master's Degree,Online Media,Management,0
7,8,Lead Guest Service Specialist,"US, CA, San Francisco",NaN,NaN,Airenvy’s mission is to provide lucrative yet ...,Who is Airenvy?Hey there! We are seasoned entr...,"Experience with CRM software, live chat, and p...",Competitive Pay. You'll be able to eat steak e...,0,1,1,NaN,NaN,NaN,NaN,NaN,0
8,9,HP BSM SME,"US, FL, Pensacola",NaN,NaN,Solutions3 is a woman-owned small business who...,Implementation/Configuration/Testing/Training ...,MUST BE A US CITIZEN.An active TS/SCI clearanc...,NaN,0,1,1,Full-time,Associate,NaN,Information Technology and Services,NaN,0
9,10,Customer Service Associate - Part Time,"US, AZ, Phoenix",NaN,NaN,"Novitex Enterprise Solutions, formerly Pitney ...",The Customer Service Associate will be based i...,Minimum Requirements:Minimum of 6 months custo...,NaN,0,1,0,Part-time,Entry level,High School or equivalent,Financial Services,Customer Service,0


In [50]:
# check for columns

df.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [51]:
# drop the job ID colum

df.drop(columns=['job_id'], inplace = True)

In [52]:
# verify

df.columns

Index(['title', 'location', 'department', 'salary_range', 'company_profile',
       'description', 'requirements', 'benefits', 'telecommuting',
       'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [53]:
# check duplicates

df.duplicated().sum()

281

In [54]:
# drop duplicates

df.drop_duplicates(inplace = True)

In [55]:
# verify

df.duplicated().sum()

0

In [56]:
# check missing values 

df.isnull().sum()

title                      0
location                 341
department             11328
salary_range           14772
company_profile         3282
description                1
requirements            2649
benefits                7108
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3428
required_experience     6962
required_education      8009
industry                4837
function                6363
fraudulent                 0
dtype: int64

In [57]:
# drop rows with missing values in key columns like title, description, company_profile

df = df.dropna(subset =['title', 'description', 'company_profile'])

In [58]:
# recheck for missing values

df.isnull().sum()

title                      0
location                 280
department              8999
salary_range           12240
company_profile            0
description                0
requirements            1901
benefits                5449
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         2486
required_experience     5455
required_education      6215
industry                3658
function                4912
fraudulent                 0
dtype: int64

In [59]:
# handle missing values for the rest columns

df['department'] = df['department'].fillna('Unknown').str.strip().str.lower()

df['location'] = df['location'].fillna('Unknown')

# salary range

df['salary_range'] = df['salary_range'].isnull().astype(int)
df['salary_range'] = df['salary_range'].fillna('Not Provided')

# handling requirements and benefits

df['requirements'] = df['requirements'].fillna('Not Specified')
df['benefits'] = df['benefits'].fillna('Not Specified')

# handling employment_type, required_experience, required_education

cat_cols = ['employment_type', 'required_experience','required_education']

for col in cat_cols:
    df[col] = df[col].str.lower().str.strip().fillna('Uknown')

# handling industry and function

df['industry'] = df['industry'].fillna('Unknown').str.lower().str.strip()
df['function'] = df['function'].fillna('Unknown').str.lower().str.strip()


In [60]:
# reconfirm missing values

df.isnull().sum()

title                  0
location               0
department             0
salary_range           0
company_profile        0
description            0
requirements           0
benefits               0
telecommuting          0
has_company_logo       0
has_questions          0
employment_type        0
required_experience    0
required_education     0
industry               0
function               0
fraudulent             0
dtype: int64

In [61]:
# validation

df.value_counts()

title                                                              location               department        salary_range  company_profile                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    description

In [62]:
# cleaning texts columns

def clean_text(text):
    if pd.isnull(text): return ""
    text = BeautifulSoup(text, "html.parser").get_text()  
    text = re.sub(r'\W+', ' ', text)  
    return text.strip().lower()

text_cols = ['description', 'requirements', 'benefits', 'company_profile']
for col in text_cols:
    df[col] = df[col].apply(clean_text)

In [63]:
# reconfirm texts cleanup

df.head(50)

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,Marketing Intern,"US, NY, New York",marketing,1,we re food52 and we ve created a groundbreakin...,food52 a fast growing james beard award winnin...,experience with content management systems a m...,not specified,0,1,0,other,internship,Uknown,unknown,marketing,0
1,Customer Service - Cloud Video Production,"NZ, , Auckland",success,1,90 seconds the worlds cloud video production s...,organised focused vibrant awesome do you have ...,what we expect from you your key responsibilit...,what you will get from usthrough being part of...,0,1,0,full-time,not applicable,Uknown,marketing and advertising,customer service,0
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",unknown,1,valor services provides workforce solutions th...,our client located in houston is actively seek...,implement pre commissioning and commissioning ...,not specified,0,1,0,Uknown,Uknown,Uknown,unknown,unknown,0
3,Account Executive - Washington DC,"US, DC, Washington",sales,1,our passion for improving quality of life thro...,the company esri environmental systems researc...,education bachelor s or master s in gis busine...,our culture is anything but corporate we have ...,0,1,0,full-time,mid-senior level,bachelor's degree,computer software,sales,0
4,Bill Review Manager,"US, FL, Fort Worth",unknown,1,spotsource solutions llc is a global human cap...,job title itemization review managerlocation f...,qualifications rn license in the state of texa...,full benefits offered,0,1,1,full-time,mid-senior level,bachelor's degree,hospital & health care,health care provider,0
6,Head of Content (m/f),"DE, BE, Berlin",androidpit,0,founded in 2009 the fonpit ag rose with its in...,your responsibilities manage the english speak...,your know how university or college degree in ...,your benefits being part of a fast growing com...,0,1,1,full-time,mid-senior level,master's degree,online media,management,0
7,Lead Guest Service Specialist,"US, CA, San Francisco",unknown,1,airenvy s mission is to provide lucrative yet ...,who is airenvy hey there we are seasoned entre...,experience with crm software live chat and pho...,competitive pay you ll be able to eat steak ev...,0,1,1,Uknown,Uknown,Uknown,unknown,unknown,0
8,HP BSM SME,"US, FL, Pensacola",unknown,1,solutions3 is a woman owned small business who...,implementation configuration testing training ...,must be a us citizen an active ts sci clearanc...,not specified,0,1,1,full-time,associate,Uknown,information technology and services,unknown,0
9,Customer Service Associate - Part Time,"US, AZ, Phoenix",unknown,1,novitex enterprise solutions formerly pitney b...,the customer service associate will be based i...,minimum requirements minimum of 6 months custo...,not specified,0,1,0,part-time,entry level,high school or equivalent,financial services,customer service,0
11,Talent Sourcer (6 months fixed-term contract),"GB, LND, London",hr,1,want to build a 21st century financial service...,transferwise is the clever new way to move mon...,we re looking for someone who proven track rec...,you will join one of europe s most hotly tippe...,0,1,0,Uknown,Uknown,Uknown,unknown,unknown,0


In [68]:
# Splitting location into city,country

df[['state_country', 'city']] = df['location'].str.split(',', n=1, expand=True)

# Strip whitespace from the new columns
df['state_country'] = df['state_country'].str.strip()
df['city'] = df['city'].str.strip()


In [69]:
# reconfrim this 

df.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,city,state_country
0,Marketing Intern,"US, NY, New York",marketing,1,we re food52 and we ve created a groundbreakin...,food52 a fast growing james beard award winnin...,experience with content management systems a m...,not specified,0,1,0,other,internship,Uknown,unknown,marketing,0,"NY, New York",US
1,Customer Service - Cloud Video Production,"NZ, , Auckland",success,1,90 seconds the worlds cloud video production s...,organised focused vibrant awesome do you have ...,what we expect from you your key responsibilit...,what you will get from usthrough being part of...,0,1,0,full-time,not applicable,Uknown,marketing and advertising,customer service,0,", Auckland",NZ
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",unknown,1,valor services provides workforce solutions th...,our client located in houston is actively seek...,implement pre commissioning and commissioning ...,not specified,0,1,0,Uknown,Uknown,Uknown,unknown,unknown,0,"IA, Wever",US
3,Account Executive - Washington DC,"US, DC, Washington",sales,1,our passion for improving quality of life thro...,the company esri environmental systems researc...,education bachelor s or master s in gis busine...,our culture is anything but corporate we have ...,0,1,0,full-time,mid-senior level,bachelor's degree,computer software,sales,0,"DC, Washington",US
4,Bill Review Manager,"US, FL, Fort Worth",unknown,1,spotsource solutions llc is a global human cap...,job title itemization review managerlocation f...,qualifications rn license in the state of texa...,full benefits offered,0,1,1,full-time,mid-senior level,bachelor's degree,hospital & health care,health care provider,0,"FL, Fort Worth",US


In [70]:
# drop location column
df.drop(columns=['location'], inplace = True)

In [71]:
# confirm if location column was dropped

df.head()

,title,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,city,state_country
0,Marketing Intern,marketing,1,we re food52 and we ve created a groundbreakin...,food52 a fast growing james beard award winnin...,experience with content management systems a m...,not specified,0,1,0,other,internship,Uknown,unknown,marketing,0,"NY, New York",US
1,Customer Service - Cloud Video Production,success,1,90 seconds the worlds cloud video production s...,organised focused vibrant awesome do you have ...,what we expect from you your key responsibilit...,what you will get from usthrough being part of...,0,1,0,full-time,not applicable,Uknown,marketing and advertising,customer service,0,", Auckland",NZ
2,Commissioning Machinery Assistant (CMA),unknown,1,valor services provides workforce solutions th...,our client located in houston is actively seek...,implement pre commissioning and commissioning ...,not specified,0,1,0,Uknown,Uknown,Uknown,unknown,unknown,0,"IA, Wever",US
3,Account Executive - Washington DC,sales,1,our passion for improving quality of life thro...,the company esri environmental systems researc...,education bachelor s or master s in gis busine...,our culture is anything but corporate we have ...,0,1,0,full-time,mid-senior level,bachelor's degree,computer software,sales,0,"DC, Washington",US
4,Bill Review Manager,unknown,1,spotsource solutions llc is a global human cap...,job title itemization review managerlocation f...,qualifications rn license in the state of texa...,full benefits offered,0,1,1,full-time,mid-senior level,bachelor's degree,hospital & health care,health care provider,0,"FL, Fort Worth",US


In [74]:
# removing short job descripion
df = df[df['description'].str.len()>50]

In [76]:
# remove where job title equal description 

df = df[df['description'] !=df['title']]

In [77]:
# saving our cleaning dataset

df.to_csv('cleaned_dake_job_posting.csv', index=False)